In [1]:
import os
from playwright.async_api import async_playwright
import pandas as pd

RESULT_CSV = "result.csv"
COLUMNS = ["title","desc","category", "website", "email", "tel"]
URL = 'https://www.office-expo.jp/tokyo-2/ja-jp/search/2024/directory.html?refinementList%5B0%5D%5B0%5D=sponsoredCategory.id%3A836631&refinementList%5B0%5D%5B1%5D=sponsoredCategory.id%3A836635&refinementList%5B0%5D%5B2%5D=sponsoredCategory.id%3A836637&refinementList%5B0%5D%5B3%5D=sponsoredCategory.id%3A836631&refinementList%5B0%5D%5B4%5D=sponsoredCategory.id%3A836635&refinementList%5B0%5D%5B5%5D=sponsoredCategory.id%3A836631&refinementList%5B0%5D%5B6%5D=exhibitorFilters.%E5%B1%95%E7%A4%BA%E4%BC%9A.lvl0%3Aid-836631&refinementList%5B0%5D%5B7%5D=exhibitorFilters.%E5%B1%95%E7%A4%BA%E4%BC%9A.lvl0%3Aid-836635&refinementList%5B0%5D%5B8%5D=exhibitorFilters.%E5%B1%95%E7%A4%BA%E4%BC%9A.lvl0%3Aid-836637'





In [2]:
def add_to_result_csv(data: list[dict[str, str]]):
    if not os.path.exists(RESULT_CSV):
        dirname = os.path.dirname(RESULT_CSV)
        if dirname:
            os.makedirs(os.path.dirname(RESULT_CSV), exist_ok=True)
        pd.DataFrame(columns=COLUMNS).to_csv(RESULT_CSV, index=False)
    
    df = pd.read_csv(RESULT_CSV)
    pd.concat([df, pd.DataFrame(data)]).to_csv(RESULT_CSV, index=False)


In [3]:

playwright = await async_playwright().start()

browser = await playwright.chromium.launch()
browser_context = await browser.new_context()
page = await browser_context.new_page()
await page.goto(URL)

<Response url='https://www.office-expo.jp/tokyo-2/ja-jp/search/2024/directory.html?refinementList%5B0%5D%5B0%5D=sponsoredCategory.id%3A836631&refinementList%5B0%5D%5B1%5D=sponsoredCategory.id%3A836635&refinementList%5B0%5D%5B2%5D=sponsoredCategory.id%3A836637&refinementList%5B0%5D%5B3%5D=sponsoredCategory.id%3A836631&refinementList%5B0%5D%5B4%5D=sponsoredCategory.id%3A836635&refinementList%5B0%5D%5B5%5D=sponsoredCategory.id%3A836631&refinementList%5B0%5D%5B6%5D=exhibitorFilters.%E5%B1%95%E7%A4%BA%E4%BC%9A.lvl0%3Aid-836631&refinementList%5B0%5D%5B7%5D=exhibitorFilters.%E5%B1%95%E7%A4%BA%E4%BC%9A.lvl0%3Aid-836635&refinementList%5B0%5D%5B8%5D=exhibitorFilters.%E5%B1%95%E7%A4%BA%E4%BC%9A.lvl0%3Aid-836637' request=<Request url='https://www.office-expo.jp/tokyo-2/ja-jp/search/2024/directory.html?refinementList%5B0%5D%5B0%5D=sponsoredCategory.id%3A836631&refinementList%5B0%5D%5B1%5D=sponsoredCategory.id%3A836635&refinementList%5B0%5D%5B2%5D=sponsoredCategory.id%3A836637&refinementList%5B0%5D%

In [9]:
cards = await page.locator(".directory-item-feature-toggled.exhibitor-category").all()
res = []
for i, card in enumerate(cards):
    title = (await card.locator("h3").inner_text()) if (await card.locator("h3").all()) else ""
    desc = (await card.locator(".exhibitor-info").inner_text()) if (await card.locator(".exhibitor-info").all()) else ""
    tags = (await card.locator(".pps-tags").inner_text()) if (await card.locator(".pps-tags").all()) else ""
    el = {
        "title": title,
        "desc": desc,
        "category": tags,
        "website": "",
        "email": "",
        "tel": ""
    }
    contacts = await card.locator("a.contact-option").all()
    for contact in contacts:
        text = (await contact.inner_text())
        if "ウェブサイト" in text:
            el["website"] = (await contact.get_attribute("href"))
        if "メール" in text:
            el["email"] = (await contact.get_attribute("href"))
        if "電話" in text:
            el["tel"] = (await contact.get_attribute("href"))
    res.append(el)
    
    if len(res) > 10:
        add_to_result_csv(res)
        res = []

add_to_result_csv(res)


https://hcm-jinjer.com/
https://www.teamspirit.com/
https://www.u-can.co.jp/houjin/
https://www.pfa.or.jp/
https://kitera-cloud.jp/
https://sys-evo.co.jp/
https://hybridbpo-syspro.net/
https://job-us.exwork.jp/
https://gvatech.co.jp/
https://newspicks.com/
https://himanager.me/
https://five-vai.com/
https://findy-code.io/enterprise-service/


In [ ]:
await browser.close()
await playwright.stop()